In [2]:
using AutomotiveDrivingModels
using AutoViz
using Reactive
using Interact
using SplineUtils
using HierarchicalRecedingHorizonController
using NearestNeighbors


# import PyPlot

/Users/kyle/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
# scene parameters
scene = Scene()
framerate = 24
Δt = 1.0/framerate # size of rendering timesteps
n_integration_sub_steps = 3 # sub steps for smoother integration
context = IntegratedContinuous(Δt, n_integration_sub_steps) # integrated continuous context


#car parameters
car_length = 4.8 # front wheel to back wheel
car_width = 2.5
v⁰  = 0.0 # initial velocity
δ⁰ = 0.0 # initical steering angle

############### INITIALIZE TRACK ################

lane_width = 8.0*DEFAULT_LANE_WIDTH
radius = 45.0
edge_buffer = 0.25
T_MAX = lane_width/2.0 - car_width/2.0 - edge_buffer # max allowable projection distance from center of lane
base_speed= 0.0

# spline control points
# Pts = 40*[0 -1 -2 -3 -3 -3 -2 -1 -1 -1 -2 -3 -4 -5 -5 -5 -5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
#          0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]
Pts = 40*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -5.5 -5 -4.5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]


degree = 3 # degree of spline
num_points = 10001

# compute B spline
T, tt, rx, ry = ClosedB_Spline(Pts, degree, num_points)
# compute B spline derivative
ṙx, ṙy = B_SplineDerivative(T,tt,Pts,degree)
# compute theta 
θ = atan2(ṙy,ṙx) # unit tangent vector
# compute arc length s
s = zeros(size(rx))
s[2:end] = cumsum(sqrt(diff(rx).^2 + diff(ry).^2))
# compute curvature
k = diff(θ)./diff(s) # curvature
# resample evenly along spline
num_samples = 420
xP, yP, θP, sP, kP = ResampleSplineEven(rx,ry,θ,s,k,num_samples)

# KDtree for projection onto roadway
# tree = KDTree([xP';yP'])

lane_width = 20.0
roadway, tree = GenSplineRoadway(xP,yP,θP,sP,kP,lane_width)

roadind1 = RoadIndex(CurveIndex(8,0),LaneTag(1,1))
roadind2 = RoadIndex(CurveIndex(5,0),LaneTag(1,1))
roadind3 = RoadIndex(CurveIndex(1,0),LaneTag(1,1))

vehstate1 = VehicleState(Frenet(roadind1, roadway), roadway, 0.0)
vehstate2 = VehicleState(Frenet(roadind2, roadway), roadway, 0.0)
vehstate3 = VehicleState(Frenet(roadind3, roadway), roadway, 0.0)

vehdef1 = VehicleDef(1,AgentClass.CAR, car_length, car_width)
vehdef2 = VehicleDef(2,AgentClass.CAR, car_length, car_width)
vehdef3 = VehicleDef(3,AgentClass.CAR, car_length, car_width)

push!(scene,Vehicle(vehstate1, vehdef1))
push!(scene,Vehicle(vehstate2, vehdef2))
push!(scene,Vehicle(vehstate3, vehdef3))

models = Dict{Int, DriverModel}()
carcolors = Dict{Int, Colorant}()
carcolors[1] = colorant"red"
carcolors[2] = colorant"blue"
carcolors[2] = colorant"yellow"


# hrhc = HRHC(1,roadway,context)
models[1] = HRHC(1,roadway,context,ΔV₊=2.05,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=115.0,V_STEPS=230,k=1)
models[2] = HRHC(2,roadway,context,ΔV₊=1.55,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=125.0,V_STEPS=250,k=1)
models[3] = HRHC(3,roadway,context,ΔV₊=2.55,ΔV₋=4.05,Δδ=Float64(π)/12,V_MIN=0.0,V_MAX=130.0,V_STEPS=250,k=2)
actions = Array(DriveAction, length(scene))

obstacleMap = generateObstacleMap(scene, models)

LoadError: UndefVarError: ClosedB_Spline not defined